# High Price Predict Model

In [13]:
# 라이브러리
import pandas as pd
kospi = pd.read_csv('./KOSPI.csv')

# KOSPI에 상장된 종목 코드
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).rjust(6, '0'))
code_list = kospi['종목코드'].unique()

# Json파일 
submission = []

# 라이브러리
import numpy as np

# 진행과정 Check
check = 0

# 코드 별 진행
for code in code_list : 

    # Hyper Parameter
    tr_name = 'TR_SCHART'
    term = 'D'
    start_date = '20180510'   
    end_date = '20200229'
    Lookup = '9999' 
    
    # url
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

    # url open
    from urllib.request import urlopen
    url_page = urlopen(url)

    # json 파일로 받아오기
    import json
    url_data = json.loads(url_page.read())

    # json을 데이터프레임으로 바꾸기
    import pandas as pd
    data = pd.DataFrame(url_data)
    
    # Data Column명 변경
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    
    # 순서 뒤집기
    data = data[::-1]
    data = data.set_index('date')
    
    # 필요한 변수만 추출
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]

    # 만약에 데이터가 비어있다면 그 종목은 PASS
    if data["open"][0] == '' : 
        continue
    
    # 변수 속성 정수로 바꾸기
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
        
    # 거래량과 거래가격을 모두 로그 변환
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    
    # 각각의 피쳐들에 대해 
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    
    # 타겟 생성 : 다음날의 고가를 예측
    data['y'] = data['high'].shift(-1)
    
    # NaN이 들어가 있는 행 제거
    data = data.dropna()
    
    # Input과 Target을 분리
    X = data[data.columns[:-1]]
    y = data['y']
    
    # Linear Regression
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    
    # Test Data 생성하기
    start_date, end_date = '20200301', '20200316'
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup
    url_page = urlopen(url)
    url_data = json.loads(url_page.read())
    data = pd.DataFrame(url_data)
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr',
                    'rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    data['y'] = data['high'].shift(-1)
    data = data.dropna()
    data = data[-1:]
    data = data[data.columns[:-1]]

    
    # 필요한것들
    # 1. 오늘 종가
    # 2. 내일 고가
    # 3. 오늘 종가 대비 내일 종가 변화율
    # 4. 종목 뽑아내기
    
    # 각 종목별 결과 출력
    print('{:>4}/{}\tCODE = {}\t{:>8}원' .format(check, len(code_list), code, int(round(reg.predict(data)[0]))))
    check += 1
    
    # 딕셔너리로 저장하기
    pred = dict({"model" : "psh_lr_high",
                  "seed" : 1234,
                  "type" : "high",
                  "code" : code,
                  "date" : "20-03-16 00:00:00",
                "prdate" : 0,
                 "value" : int(round(reg.predict(data)[0]))})
    
    # 제출 리스트에 추가하기
    submission.append(pred)

   0/2303	CODE = 060310	    2086원
   1/2303	CODE = 095570	    3560원
   2/2303	CODE = 068400	    7898원
   3/2303	CODE = 006840	   19389원
   4/2303	CODE = 054620	    6965원
   5/2303	CODE = 265520	   25556원
   6/2303	CODE = 211270	    6066원
   7/2303	CODE = 027410	    3846원
   8/2303	CODE = 282330	  136644원
   9/2303	CODE = 138930	    4879원
  10/2303	CODE = 001460	  178508원
  11/2303	CODE = 001040	   62006원
  12/2303	CODE = 079160	   21881원
  13/2303	CODE = 035760	  105040원
  14/2303	CODE = 000120	  123188원
  15/2303	CODE = 011150	    3061원
  16/2303	CODE = 097950	  223322원
  17/2303	CODE = 051500	   16498원
  18/2303	CODE = 037560	    3259원
  19/2303	CODE = 058820	    3269원
  20/2303	CODE = 023460	    1163원
  21/2303	CODE = 065770	    1922원
  22/2303	CODE = 083660	    4170원
  23/2303	CODE = 000590	   49065원
  24/2303	CODE = 012030	     625원
  25/2303	CODE = 306620	    1674원
  26/2303	CODE = 322780	    2046원
  27/2303	CODE = 016610	    3033원
  28/2303	CODE = 005830	   30389원
  29/2303	CODE

 241/2303	CODE = 066620	   11662원
 242/2303	CODE = 043650	    3595원
 243/2303	CODE = 006050	    2368원
 244/2303	CODE = 060480	    3893원
 245/2303	CODE = 078130	    4593원
 246/2303	CODE = 002720	    4529원
 247/2303	CODE = 204020	    1713원
 248/2303	CODE = 083420	    2620원
 249/2303	CODE = 186230	    6683원
 250/2303	CODE = 014530	    2651원
 251/2303	CODE = 900070	     537원
 252/2303	CODE = 204620	    2893원
 253/2303	CODE = 019660	    2334원
 254/2303	CODE = 014280	    2327원
 255/2303	CODE = 053260	    3069원
 256/2303	CODE = 008870	   48847원
 257/2303	CODE = 001570	    2467원
 258/2303	CODE = 002990	    6709원
 259/2303	CODE = 011780	   54376원
 260/2303	CODE = 214330	    2667원
 261/2303	CODE = 001210	    2292원
 262/2303	CODE = 073240	    3316원
 263/2303	CODE = 036190	   22253원
 264/2303	CODE = 049080	     873원
 265/2303	CODE = 035460	    2170원
 266/2303	CODE = 092440	    2899원
 267/2303	CODE = 000270	   29568원
 268/2303	CODE = 024110	    7306원
 269/2303	CODE = 013700	    1019원
 270/2303	CODE

 482/2303	CODE = 049770	  171273원
 483/2303	CODE = 013120	    3396원
 484/2303	CODE = 018500	     740원
 485/2303	CODE = 006040	  177674원
 486/2303	CODE = 030720	    6881원
 487/2303	CODE = 014820	   20196원
 488/2303	CODE = 163560	    4409원
 489/2303	CODE = 109860	   10515원
 490/2303	CODE = 032960	    9471원
 491/2303	CODE = 004890	   44102원
 492/2303	CODE = 002690	    1573원
 493/2303	CODE = 023790	    2252원
 494/2303	CODE = 005290	   13110원
 495/2303	CODE = 025900	   12593원
 496/2303	CODE = 000020	    6230원
 497/2303	CODE = 000150	   37428원
 498/2303	CODE = 011160	     924원
 499/2303	CODE = 241560	   21929원
 500/2303	CODE = 042670	    3217원
 501/2303	CODE = 034020	    3285원
 502/2303	CODE = 016740	    2143원
 503/2303	CODE = 078590	    2603원
 504/2303	CODE = 073190	    2205원
 505/2303	CODE = 030350	    1682원
 506/2303	CODE = 203650	    1538원
 507/2303	CODE = 060570	    4216원
 508/2303	CODE = 192650	    4304원
 509/2303	CODE = 217620	    1179원
 510/2303	CODE = 187870	   12064원
 511/2303	CODE

 723/2303	CODE = 066980	     554원
 724/2303	CODE = 064480	    2325원
 725/2303	CODE = 251630	    6811원
 726/2303	CODE = 263920	    3670원
 727/2303	CODE = 033560	    3426원
 728/2303	CODE = 121800	    5403원
 729/2303	CODE = 148140	    6700원
 730/2303	CODE = 100220	    6511원
 731/2303	CODE = 200780	   17796원
 732/2303	CODE = 141000	    7303원
 733/2303	CODE = 090460	   16879원
 734/2303	CODE = 083650	    1918원
 735/2303	CODE = 086670	    5811원
 736/2303	CODE = 138580	    9833원
 737/2303	CODE = 082920	   13902원
 738/2303	CODE = 054220	     485원
 739/2303	CODE = 042370	    5078원
 740/2303	CODE = 032850	    4565원
 741/2303	CODE = 101140	    3245원
 742/2303	CODE = 238200	   16992원
 743/2303	CODE = 093190	    4687원
 744/2303	CODE = 065450	    2343원
 745/2303	CODE = 005180	   48564원
 746/2303	CODE = 072950	    4323원
 747/2303	CODE = 143240	   22585원
 748/2303	CODE = 003960	   11112원
 749/2303	CODE = 008040	     700원
 750/2303	CODE = 007160	   21837원
 751/2303	CODE = 014710	    3477원
 752/2303	CODE

 964/2303	CODE = 031430	  200114원
 965/2303	CODE = 031440	   44580원
 966/2303	CODE = 006880	    2356원
 967/2303	CODE = 056000	    4168원
 968/2303	CODE = 002800	    4468원
 969/2303	CODE = 307180	    1242원
 970/2303	CODE = 323280	    2046원
 971/2303	CODE = 005800	   80659원
 972/2303	CODE = 001720	   46873원
 973/2303	CODE = 009270	    1258원
 974/2303	CODE = 017000	    3061원
 975/2303	CODE = 002700	    1637원
 976/2303	CODE = 012790	    6009원
 977/2303	CODE = 138070	    4692원
 978/2303	CODE = 019170	    8582원
 979/2303	CODE = 002870	    1019원
 980/2303	CODE = 005450	    4945원
 981/2303	CODE = 293940	    6828원
 982/2303	CODE = 277480	    2051원
 983/2303	CODE = 323230	    2028원
 984/2303	CODE = 055550	   26057원
 985/2303	CODE = 001770	   20900원
 986/2303	CODE = 056700	    2498원
 987/2303	CODE = 187270	    1660원
 988/2303	CODE = 004080	    8808원
 989/2303	CODE = 243840	   34029원
 990/2303	CODE = 108320	   30873원
 991/2303	CODE = 222800	    9622원
 992/2303	CODE = 036710	    1153원
 993/2303	CODE

1205/2303	CODE = 273060	    2048원
1206/2303	CODE = 310840	    2048원
1207/2303	CODE = 319400	    2017원
1208/2303	CODE = 181710	   66213원
1209/2303	CODE = 208860	    3125원
1210/2303	CODE = 183490	   50984원
1211/2303	CODE = 085310	     769원
1212/2303	CODE = 182400	    7342원
1213/2303	CODE = 009810	     658원
1214/2303	CODE = 206400	     869원
1215/2303	CODE = 069410	    8134원
1216/2303	CODE = 004250	    3285원
1217/2303	CODE = 048830	    1249원
1218/2303	CODE = 096870	    2303원
1219/2303	CODE = 900140	    4096원
1220/2303	CODE = 061970	    6210원
1221/2303	CODE = 138690	    6342원
1222/2303	CODE = 290650	   41945원
1223/2303	CODE = 066970	   21930원
1224/2303	CODE = 156100	    7010원
1225/2303	CODE = 073110	    9098원
1226/2303	CODE = 083310	    9279원
1227/2303	CODE = 037950	    2335원
1228/2303	CODE = 170920	    7732원
1229/2303	CODE = 058630	    3092원
1230/2303	CODE = 019590	    1160원
1231/2303	CODE = 097520	   30758원
1232/2303	CODE = 179290	    2455원
1233/2303	CODE = 009780	    4070원
1234/2303	CODE

1446/2303	CODE = 102710	   21895원
1447/2303	CODE = 094190	     787원
1448/2303	CODE = 063760	    6621원
1449/2303	CODE = 123570	    2618원
1450/2303	CODE = 095190	    2880원
1451/2303	CODE = 091120	    6854원
1452/2303	CODE = 102460	   11345원
1453/2303	CODE = 039030	   66352원
1454/2303	CODE = 088290	    7870원
1455/2303	CODE = 084680	    3598원
1456/2303	CODE = 181340	    3096원
1457/2303	CODE = 035810	    3606원
1458/2303	CODE = 090850	    7927원
1459/2303	CODE = 099750	   13697원
1460/2303	CODE = 092130	   16827원
1461/2303	CODE = 016250	   39130원
1462/2303	CODE = 096040	     131원
1463/2303	CODE = 134060	    5374원
1464/2303	CODE = 001840	    3634원
1465/2303	CODE = 000760	   12248원
1466/2303	CODE = 024810	     143원
1467/2303	CODE = 014990	    1265원
1468/2303	CODE = 041830	   16475원
1469/2303	CODE = 079950	    1117원
1470/2303	CODE = 277410	    1333원
1471/2303	CODE = 060150	    6793원
1472/2303	CODE = 033230	    3044원
1473/2303	CODE = 006490	    2287원
1474/2303	CODE = 037330	    1430원
1475/2303	CODE

1687/2303	CODE = 078940	    2979원
1688/2303	CODE = 080530	    3589원
1689/2303	CODE = 224060	     386원
1690/2303	CODE = 078650	    1212원
1691/2303	CODE = 104540	    5630원
1692/2303	CODE = 027050	    3968원
1693/2303	CODE = 007810	    8728원
1694/2303	CODE = 101670	     809원
1695/2303	CODE = 123410	    2041원
1696/2303	CODE = 152330	   11950원
1697/2303	CODE = 003690	    7228원
1698/2303	CODE = 036690	    2608원
1699/2303	CODE = 049430	    7322원
1700/2303	CODE = 183300	   26888원
1701/2303	CODE = 041960	   17742원
1702/2303	CODE = 089890	    4911원
1703/2303	CODE = 009730	     394원
1704/2303	CODE = 192820	   65611원
1705/2303	CODE = 044820	   10541원
1706/2303	CODE = 222040	    4692원
1707/2303	CODE = 241710	    7432원
1708/2303	CODE = 005070	    8064원
1709/2303	CODE = 005420	    6708원
1710/2303	CODE = 069110	    4756원
1711/2303	CODE = 071950	     865원
1712/2303	CODE = 166480	   13004원
1713/2303	CODE = 045970	    5868원
1714/2303	CODE = 029960	    7198원
1715/2303	CODE = 002020	   12026원
1716/2303	CODE

1928/2303	CODE = 034950	   56876원
1929/2303	CODE = 010040	    2497원
1930/2303	CODE = 025540	   25546원
1931/2303	CODE = 222980	    3956원
1932/2303	CODE = 318000	    4032원
1933/2303	CODE = 004090	   86730원
1934/2303	CODE = 025550	    2088원
1935/2303	CODE = 002200	   15046원
1936/2303	CODE = 002960	  222478원
1937/2303	CODE = 023890	   44130원
1938/2303	CODE = 017890	   10942원
1939/2303	CODE = 080720	   11165원
1940/2303	CODE = 123890	    2108원
1941/2303	CODE = 015760	   19622원
1942/2303	CODE = 063570	    5529원
1943/2303	CODE = 041460	    2896원
1944/2303	CODE = 006200	     425원
1945/2303	CODE = 101680	    2508원
1946/2303	CODE = 039740	    2214원
1947/2303	CODE = 053300	    2774원
1948/2303	CODE = 025770	    6247원
1949/2303	CODE = 281410	    2053원
1950/2303	CODE = 291210	    2033원
1951/2303	CODE = 310870	    2042원
1952/2303	CODE = 002300	   13017원
1953/2303	CODE = 009540	   83382원
1954/2303	CODE = 023350	    3252원
1955/2303	CODE = 025890	     869원
1956/2303	CODE = 000970	    8364원
1957/2303	CODE

### 결과 송신

In [16]:
# import requests
# import json

# def send_predicts(predict):
#     payload = json.dumps(predict)
#     print(payload)
#     url = "https://lab.donutz.co/tensors"
#     headers = {'Content-Type': 'application/json; charset=utf-8'}
#     #response = requests.get(url, params=paramDict)
#     response = requests.post(url, data=payload, headers=headers, verify=False)
#     code = response.status_code
#     res = response.text
#     return (code, res)

# predict = submission
# print(send_predicts(predict))

[{"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "060310", "date": "20-03-16 00:00:00", "prdate": 0, "value": 2086}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "095570", "date": "20-03-16 00:00:00", "prdate": 0, "value": 3560}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "068400", "date": "20-03-16 00:00:00", "prdate": 0, "value": 7898}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "006840", "date": "20-03-16 00:00:00", "prdate": 0, "value": 19389}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "054620", "date": "20-03-16 00:00:00", "prdate": 0, "value": 6965}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "265520", "date": "20-03-16 00:00:00", "prdate": 0, "value": 25556}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "211270", "date": "20-03-16 00:00:00", "prdate": 0, "value": 6066}, {"model": "psh_lr_high", "seed": 1234, "type": "high", "code": "027410", "date":

/home/nvadmin/snap/jupyter/common/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lab.donutz.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


(200, '{"data":2144}')
